In [1]:
import pandas as pd # Importa biblioteca Pandas

In [2]:
# Importa as faturas SP e Faturas MG
sp = pd.read_excel(r'N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\02_Faturas\Faturas_New.xlsx', sheet_name='Mestre')
mg = pd.read_excel(r'N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\02_Faturas\Faturas MG.xlsx', sheet_name = 'Mestre')

In [3]:
# Dropa as colunas não utilizaveis dos 2 Dataframes
dropa_sp = ['Fatura', 'Resp','Data de Emissão',	'Comentários',	'Data de Verif',	'Diferença %', 'Status'	] #'Unnamed: 14', 'Unnamed: 15, 'Data de Emissão''
dropa_mg = ['Fatura', 'Resp', 'Data de Emissão','Coluna1', 'data de conferência',	'Diferença %',	'Status']

In [4]:
# Dropa as colunas não utilizaveis dos 2 Dataframes
sp = sp.drop(columns = dropa_sp)
mg = mg.drop(columns = dropa_mg)


In [5]:
# Cria a coluna Origem para identificar de onde a Nota Fiscal eh oriunda. 
sp['Origem'] = 'Faturas SP'
mg['Origem'] = 'Faturas MG'

In [6]:
#Concatena os 2 dataframes em um só.
faturas = pd.concat([mg, sp], ignore_index=True)
faturas

,Transportadora,NF,Cobrado,Recebido,Diferença,Motivo,Novo Cobrado,Coluna2,Origem,Custo Pós venda
0,Movvi,8,124.82,209.513514,84.693514,Promo Frete,NaN,NaN,Faturas MG,NaN
1,Movvi,12321,80.00,0.000000,-80.000000,Frete Interno,NaN,NaN,Faturas MG,NaN
2,Movvi,11,74.98,0.000000,-74.980000,Promo Frete,NaN,NaN,Faturas MG,NaN
3,Movvi,9,246.16,218.981982,-27.178018,NaN,NaN,NaN,Faturas MG,NaN
4,Movvi,10,61.56,62.936937,1.376937,NaN,NaN,NaN,Faturas MG,NaN
...,...,...,...,...,...,...,...,...,...,...
65578,Correios,115737,149.36,128.126126,-21.233874,NaN,NaN,NaN,Faturas SP,NaN
65579,Movvi,115667,127.40,105.765766,-21.634234,NaN,NaN,NaN,Faturas SP,NaN
65580,Tecmar,115599,165.59,137.297297,-28.292703,NaN,NaN,NaN,Faturas SP,NaN
65581,Alfa Transportes,115646,256.89,201.891892,-54.998108,Erro no Cálculo,NaN,NaN,Faturas SP,NaN


In [7]:
# Função que indetifica as Taxas de Zona Rural e demais taxas.
def identifica(motivo):
    taxas = ["TDE", "TZR", "TDA"]
    return motivo in taxas

In [8]:
# Criação das colunas referente a 'Promo Frete' e 'Taxas'.
faturas["Promo"] = faturas.apply(lambda row: row["Cobrado"] if row["Motivo"] == "Promo Frete" else None, axis=1)
faturas["Taxas"] = faturas.apply(lambda row: row["Cobrado"] if identifica(row["Motivo"]) else None, axis=1) # Chama a função Indentifica para encontrar as taxas.
faturas

,Transportadora,NF,Cobrado,Recebido,Diferença,Motivo,Novo Cobrado,Coluna2,Origem,Custo Pós venda,Promo,Taxas
0,Movvi,8,124.82,209.513514,84.693514,Promo Frete,NaN,NaN,Faturas MG,NaN,124.82,NaN
1,Movvi,12321,80.00,0.000000,-80.000000,Frete Interno,NaN,NaN,Faturas MG,NaN,NaN,NaN
2,Movvi,11,74.98,0.000000,-74.980000,Promo Frete,NaN,NaN,Faturas MG,NaN,74.98,NaN
3,Movvi,9,246.16,218.981982,-27.178018,NaN,NaN,NaN,Faturas MG,NaN,NaN,NaN
4,Movvi,10,61.56,62.936937,1.376937,NaN,NaN,NaN,Faturas MG,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
65578,Correios,115737,149.36,128.126126,-21.233874,NaN,NaN,NaN,Faturas SP,NaN,NaN,NaN
65579,Movvi,115667,127.40,105.765766,-21.634234,NaN,NaN,NaN,Faturas SP,NaN,NaN,NaN
65580,Tecmar,115599,165.59,137.297297,-28.292703,NaN,NaN,NaN,Faturas SP,NaN,NaN,NaN
65581,Alfa Transportes,115646,256.89,201.891892,-54.998108,Erro no Cálculo,NaN,NaN,Faturas SP,NaN,NaN,NaN


In [9]:
# Agrupa todas as notas fiscais do dataframe para não ter Relacionamento de (*:*) com a base Mestre.
soma = ['Transportadora', 'Cobrado', 'Recebido ','Diferença', 'Motivo', 'Promo', 'Novo Cobrado','Taxas', 'Origem' ]
faturas = faturas.groupby('NF')[soma].sum().reset_index()
faturas

,NF,Transportadora,Cobrado,Recebido,Diferença,Motivo,Promo,Novo Cobrado,Taxas,Origem
0,0,RodomaxLog,286.34,0.000000,-286.340000,0,0.00,0,0.0,Faturas SP
1,1,Gran Cargo,141.47,0.000000,-141.470000,Garantia,0.00,0,0.0,Faturas SP
2,2,Correios,602.51,0.000000,-602.510000,Retorno,0.00,0,0.0,Faturas SP
3,8,MovviMovvi,249.64,209.513514,-40.126486,Promo FreteRetorno,124.82,0,0.0,Faturas MGFaturas MG
4,9,Movvi,246.16,218.981982,-27.178018,0,0.00,0,0.0,Faturas MG
...,...,...,...,...,...,...,...,...,...,...
63787,8281139,Tecmar Transportes,194.07,0.000000,-194.070000,Garantia,0.00,0,0.0,Faturas SP
63788,17305\n,Solística,114.80,0.000000,-114.800000,Retorno,0.00,0,0.0,Faturas SP
63789,4937724-890,Exata Cargo,337.96,0.000000,-337.960000,Garantia,0.00,0,0.0,Faturas SP
63790,86243,Gran Cargo,106.80,199.500000,92.700000,0,0.00,0,0.0,Faturas SP


In [10]:
# Tratamento preguiçoso da coluna Origem.
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SPFaturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SPFaturas SPFaturas SPFaturas SP', 'Faturas SP', case=False)
faturas

,NF,Transportadora,Cobrado,Recebido,Diferença,Motivo,Promo,Novo Cobrado,Taxas,Origem
0,0,RodomaxLog,286.34,0.000000,-286.340000,0,0.00,0,0.0,Faturas SP
1,1,Gran Cargo,141.47,0.000000,-141.470000,Garantia,0.00,0,0.0,Faturas SP
2,2,Correios,602.51,0.000000,-602.510000,Retorno,0.00,0,0.0,Faturas SP
3,8,MovviMovvi,249.64,209.513514,-40.126486,Promo FreteRetorno,124.82,0,0.0,Faturas MG
4,9,Movvi,246.16,218.981982,-27.178018,0,0.00,0,0.0,Faturas MG
...,...,...,...,...,...,...,...,...,...,...
63787,8281139,Tecmar Transportes,194.07,0.000000,-194.070000,Garantia,0.00,0,0.0,Faturas SP
63788,17305\n,Solística,114.80,0.000000,-114.800000,Retorno,0.00,0,0.0,Faturas SP
63789,4937724-890,Exata Cargo,337.96,0.000000,-337.960000,Garantia,0.00,0,0.0,Faturas SP
63790,86243,Gran Cargo,106.80,199.500000,92.700000,0,0.00,0,0.0,Faturas SP


In [11]:
faturas = faturas.iloc[:-4]
faturas

,NF,Transportadora,Cobrado,Recebido,Diferença,Motivo,Promo,Novo Cobrado,Taxas,Origem
0,0,RodomaxLog,286.34,0.000000,-286.340000,0,0.00,0,0.0,Faturas SP
1,1,Gran Cargo,141.47,0.000000,-141.470000,Garantia,0.00,0,0.0,Faturas SP
2,2,Correios,602.51,0.000000,-602.510000,Retorno,0.00,0,0.0,Faturas SP
3,8,MovviMovvi,249.64,209.513514,-40.126486,Promo FreteRetorno,124.82,0,0.0,Faturas MG
4,9,Movvi,246.16,218.981982,-27.178018,0,0.00,0,0.0,Faturas MG
...,...,...,...,...,...,...,...,...,...,...
63783,139694,Tecmar Transportes,166.73,0.000000,-166.730000,Retorno,0.00,0,0.0,Faturas SP
63784,147338,Gran Cargo,430.00,0.000000,-430.000000,Frete Interno,0.00,0,0.0,Faturas MG
63785,349728,Tecmar Transportes,113.89,0.000000,-113.890000,Retorno,0.00,0,0.0,Faturas MG
63786,4980563,Mira,248.58,0.000000,-248.580000,Retorno,0.00,0,0.0,Faturas SP


In [13]:
#Salva o arquivo
#faturas.to_excel(r'N:\Drives compartilhados\22 - BI\BI Operações\Logística\Custo de Frete\Faturas.xlsx', index = False) # Salva Faturas para o Bi tradicional
faturas.to_excel(r'N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2.Frete\2.Frete_Transportadora.xlsx', index = False) # Salva Faturas para o Data Product